##**Identificación de textos atípicos**

#### Identificación de textos atípicos en entre los registros de trabajo con tareas pendientes.
Se busca identificar registros atípicos (outliers) entre los textos de los registros de trabajo con tareas pendientes con el fin de mejorar el desempeño de los modelos no supervisados de clasificación, que buscarán identificas los activos afectados para los cuales se han dejado tareas pendientes.

**Preparar notebook**

In [ ]:
## Conectar el notebook a googledrive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix

**Cargar los Datos Pendientes**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_pendiente.csv', encoding='latin-1')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df = pd.read_csv('content/datos_pendiente.csv', encoding='utf-8')

**Preparación de los datos**

In [ ]:
## Recuperar los tokes (como lista) que se extrajeron en el textprep y que al exportar el archivo .csv se cargaron como cadenas de texto
df['tokens_proc'] = df['tokens_proc'].apply(lambda x: re.sub('[\[\]\']+', '', str(x)))
df['tokens_proc'] = df['tokens_proc'].apply(lambda x: x.split(', '))

In [ ]:
# Definición para pasar una lista de token a un texto
def list_to_text (lista):
  text = ' '.join(lista)
  return text

In [ ]:
#aplicamos la funcion list_to_text
df['text'] = df['tokens_proc'].apply(lambda x: list_to_text(x))

In [ ]:
#se crea lista de textos 
texts = df['text'].tolist()

**Representación vectorial de los textos**

In [ ]:
# crear representación binaria del bow 

bitvector = CountVectorizer(max_features = 500, binary = True) # se seleccionan solo 1000 dimensiones
features_bv = bitvector.fit_transform(texts)
df_bitvector=pd.DataFrame(features_bv.todense(),columns=bitvector.get_feature_names())

In [ ]:
features = df[['WORKLOGID']].merge(df_bitvector, left_on=None, right_on=None, left_index=True , right_index=True)
print(features.shape)
features.head()

(10398, 501)


,WORKLOGID,abajo,abeja,abierta,abierto,abre,abren,abrio,abrir,acceso,...,vehiculo,ver,verifica,verificar,verifico,via,viento,vientos,voltaje,zona
0,2547376,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2457599,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2789392,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,2857541,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2437852,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
features = features[features.iloc[:, 1: ].sum(axis = 1) != 0].reset_index(drop=True)
print(features.shape)

(10398, 501)


In [ ]:
#Exportar el listado de los features (variables)
#features.to_csv('Datos/features_para_revisar.csv')

In [ ]:
Y = features['WORKLOGID']

In [ ]:
X = features.iloc[:, 1:].values

**Identificación de registro más común y de registros atípicos** 

*Calculo de distancias entre registros*

* Se usa la distancia de Person
$$
d = 1-\frac{(a*b-b*c)}{\sqrt{(a+c)(b+d)(a+b)(c+d)}}
$$

In [ ]:
n = X.shape[0]
distancia = []
for i in range(n):
  temp = 0
  for j in range(n):
    d, b, c, a = confusion_matrix(X[i], X[j]).ravel()
    s = (a*d-b*c)/(np.sqrt((a+c)*(b+d)*(a+b)*(c+d)))
    dist=1-s
    temp = temp + dist
  distancia.append(temp)
distancia[:10]

[8922.297967168557,
 8800.349942610024,
 7877.11235072992,
 8569.923321991759,
 8491.217609163748,
 8022.0469932376245,
 7464.44922587804,
 8261.543249369826,
 7214.4658642062495,
 7587.669803012887]

Vector con la agregación de las distancias de cada registro a los restantes 

In [ ]:
vec_dist = np.array(distancia)

In [ ]:
vec_dist

array([8922.29796717, 8800.34994261, 7877.11235073, ..., 8050.28112504,
       7817.03844221, 7738.88008541])

In [ ]:
df_vec_dist = pd.DataFrame(vec_dist, columns=['distancia'])
df_vec_dist

,distancia
0,8922.297967
1,8800.349943
2,7877.112351
3,8569.923322
4,8491.217609
...,...
10393,8493.146201
10394,8335.218537
10395,8050.281125
10396,7817.038442


In [ ]:
Y

0        2547376
1        2457599
2        2789392
3        2857541
4        2437852
          ...   
10393    3023936
10394    3108241
10395    3038792
10396    3070258
10397    3104522
Name: WORKLOGID, Length: 10398, dtype: int64

In [ ]:
ids=pd.DataFrame(Y)
ids

,WORKLOGID
0,2547376
1,2457599
2,2789392
3,2857541
4,2437852
...,...
10393,3023936
10394,3108241
10395,3038792
10396,3070258


In [ ]:
distancias = ids[['WORKLOGID']].merge(df_vec_dist, left_on=None, right_on=None, left_index=True , right_index=True)
distancias

,WORKLOGID,distancia
0,2547376,8922.297967
1,2457599,8800.349943
2,2789392,7877.112351
3,2857541,8569.923322
4,2437852,8491.217609
...,...,...
10393,3023936,8493.146201
10394,3108241,8335.218537
10395,3038792,8050.281125
10396,3070258,7817.038442


Exportación de cada uno los registros con tareas pendeientes (identificación) con sus distancias a los registros restantes.
Esta información servirá para depurar el conjunto de datos que será sometido al modelo no supervisado.

In [ ]:
#extraemos matriz con distancias para detectar los outliers de los datos pendientes.
#distancias.to_csv('Datos/distancias_pendientes.csv')

In [ ]:
vr, pos_median = (np.min(vec_dist), np.argmin(vec_dist))
vr, pos_median

(6090.972369909832, 2876)

In [ ]:
vr, pos_ext = (np.max(vec_dist), np.argmax(vec_dist))
vr, pos_ext

(9546.38317290413, 4065)

Identificación del texto mediano (índice del registro)

In [ ]:
Y[pos_median]

2660435

* Definición del límite para identificación de outliers
* Se descartarán los registros cuya distancia a los demás esté por encima del 90% de los registros. 

In [ ]:
limite = np.percentile(vec_dist, 90)
limite

8604.775329458535

In [ ]:
result = np.where(vec_dist > limite)

In [ ]:
result[0]

array([    0,     1,    11, ..., 10375, 10385, 10386], dtype=int64)

In [ ]:
wlogid = []
for i in result[0]:
  id = Y[i]
  wlogid.append(id)

In [ ]:
datos_proc = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/datos_procesados.csv', encoding='latin-1')

#Habilitar la siguiente línea para correr el proyecto de manera local
#datos_proc = pd.read_csv('content/datos_procesados.csv', encoding='latin-1')
datos_proc.shape

(156792, 7)

In [ ]:
datos_proc = datos_proc[~datos_proc['WORKLOGID'].isin(wlogid)]
datos_proc.shape

(155752, 7)

In [ ]:
datos_proc.head()

,Unnamed: 0,RESUMEN,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,0,CIERREOT,2754950,Se revisa y se encuentra poste de 12 MTS de ji...,se revisa y se encuentra poste de metros de ji...,"['se', 'revisa', 'y', 'se', 'encuentra', 'post...","['revisa', 'poste', 'reventado', 'vehiculo', '..."
1,1,CIERREOT,3005990,Localizar falla circuito primario sub guayabal...,localizar falla circuito primario subestacion ...,"['localizar', 'falla', 'circuito', 'primario',...","['localizar', 'falla', 'circuito', 'primario',..."
2,2,CIERREOT,2634320,Se retiraron 5 ramas de Palma que estaban sobr...,se retiraron arbol de palma que estaban sobre ...,"['se', 'retiraron', 'arbol', 'de', 'palma', 'q...","['retiraron', 'arbol', 'palma', 'aisladero', '..."
3,3,CIERREOT,2634321,R27-07 se visita Cll57c#34-36 revisiÃ³n por ca...,r se visita cll c revision por calidad del ser...,"['r', 'se', 'visita', 'cll', 'c', 'revision', ...","['cll', 'revision', 'calidad', 'inspecciona', ..."
4,4,CIERREOT,2742512,Con el evento Mar 4879178 en carrera 33 por 29...,con el evento mar en carrera por c del r donde...,"['con', 'el', 'evento', 'mar', 'en', 'carrera'...","['mar', 'carrera', 'trabajador', 'reparan', 'l..."
